In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
alljobssoups = []
alljoburls = []
alljoburlstems = []

#Determine how many pages of ads are on https://jobs.chronicle.com/jobs
#When collected, was 665, now 852
#For up-to-minute, use RSS feed

for i in range(1, 665):
    #print(i)
    page=requests.get('https://jobs.chronicle.com/jobs/'+str(i)+'/')
    soup = BeautifulSoup(page.text, 'html.parser')
    alljobssoups.append(soup)\
    for a in soup.find_all('a', attrs={'class': 'js-clickable-area-link'}):
        alljoburls.append('https://jobs.chronicle.com'+str(a.get('href').replace(" \r\n\t", "").replace("\r\n\r\n\r\n\r\n", "")))
        alljoburlstems.append(a.get('href').replace(" \r\n\t", "").replace("\r\n\r\n\r\n\r\n", ""))
    print(str(i)+' is done. '+str(664-i)+' are left.')
    time.sleep(0.5)

In [ ]:
alldescsoups = []
alldesctitle = []
employ = []
loc = []
salary = []
date = []
categories = []
level = []
employtype = []
text = []

In [ ]:
#Check length of alljoburls list for range
for i in range(0,13267):
    page = requests.get(alljoburls[i])
    soup = BeautifulSoup(page.text, 'html.parser')
    alldescsoups.append(soup)
    #number of 'dd' tags determines number of job info fields. 7 is max available fields
    if len(soup.find_all('dd')) == 7:
        employ.append(soup.find_all('dd')[0].get_text().replace("\n", ""))
        loc.append(soup.find_all('dd')[1].get_text().replace("\n", ""))
        salary.append(soup.find_all('dd')[2].get_text().replace("\n", ""))
        date.append(soup.find_all('dd')[3].get_text().replace("\n", ""))
        categories.append(soup.find_all('dd')[4].get_text().replace("\r", "").replace("\t", "").replace(", \n\n", "; ").replace("\n", ""))
        level.append(soup.find_all('dd')[5].get_text().replace("\n", ""))
        employtype.append(soup.find_all('dd')[6].get_text().replace("\n", ""))
        text.append(soup.find_all(attrs = {'class': 'block fix-text job-description'}))
        for a in soup.find_all('h1'):
            alldesctitle.append(a.get_text())
    #'dd' == 6 is when the "level" field is missing, which is usually for admin positions
    elif len(soup.find_all('dd')) == 6:
        employ.append(soup.find_all('dd')[0].get_text().replace("\n", ""))
        loc.append(soup.find_all('dd')[1].get_text().replace("\n", ""))
        salary.append(soup.find_all('dd')[2].get_text().replace("\n", ""))
        date.append(soup.find_all('dd')[3].get_text().replace("\n", ""))
        categories.append(soup.find_all('dd')[4].get_text().replace("\r", "").replace("\t", "").replace(", \n\n", "; ").replace("\n", ""))
        level.append('Not Applicable')
        employtype.append(soup.find_all('dd')[5].get_text().replace("\n", ""))
        text.append(soup.find_all(attrs = {'class': 'block fix-text job-description'}))
        for a in soup.find_all('h1'):
            alldesctitle.append(a.get_text())
    #Any other cases are treated as an error
    else:
        employ.append('error')
        loc.append('error')
        salary.append('error')
        date.append('error')
        categories.append('error')
        level.append('error')
        employtype.append('error')
        text.append(soup.find_all(attrs = {'class': 'block fix-text job-description'}))
        for a in soup.find_all('h1'):
            alldesctitle.append(a.get_text())
    #Progress statement
    print(str(i+1)+' is done.'+str(13267-i+1)+' remain.')
    #Delayed to not overly tax server
    time.sleep(0.5)

In [ ]:
#Build one DataFrame from all lists. Will throw an error here if something is wrong.
alljobdat = pd.DataFrame({'title': alldesctitle, 'employer': employ, 'location': loc, 'salary': salary, 'post_date': date, 'categories': categories, 'pos_type': level, 'employ_type': employtype, 'description': text, 'url': alljoburls})
#Export CSV with data
alljobdat.to_csv('chronicle_jobs_utf8.csv', index=False, encoding='utf-8')